In [12]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'brData10ClassSimple.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Test set (1000, 28, 28) (1000,)


In [3]:
# reformat the data to flat matrix and one hot encodings

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
# first solution: simple gradient decsent and multinomial logistic regression
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session: 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. 
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 0.225884
Training accuracy: 9.8%
Loss at step 100: 0.031637
Training accuracy: 9.8%
Loss at step 200: 0.012354
Training accuracy: 9.8%
Loss at step 300: 0.007370
Training accuracy: 9.8%
Loss at step 400: 0.005198
Training accuracy: 9.8%
Loss at step 500: 0.003998
Training accuracy: 9.8%
Loss at step 600: 0.003242
Training accuracy: 9.8%
Loss at step 700: 0.002724
Training accuracy: 9.8%
Loss at step 800: 0.002347
Training accuracy: 9.8%
Test accuracy: 10.0%


In [6]:
# testing stochastic gradient descent 
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 0.179889
Minibatch accuracy: 7.8%
Minibatch loss at step 500: 0.004856
Minibatch accuracy: 10.2%
Minibatch loss at step 1000: 0.001959
Minibatch accuracy: 100.0%
Minibatch loss at step 1500: 0.001861
Minibatch accuracy: 8.6%
Minibatch loss at step 2000: 0.001581
Minibatch accuracy: 13.3%
Minibatch loss at step 2500: 0.001331
Minibatch accuracy: 14.1%
Minibatch loss at step 3000: 0.000599
Minibatch accuracy: 10.2%
Minibatch loss at step 3500: 0.000453
Minibatch accuracy: 7.0%
Minibatch loss at step 4000: 0.000650
Minibatch accuracy: 10.2%
Minibatch loss at step 4500: 0.000806
Minibatch accuracy: 10.2%
Minibatch loss at step 5000: 0.000861
Minibatch accuracy: 10.9%
Minibatch loss at step 5500: 0.000661
Minibatch accuracy: 10.2%
Minibatch loss at step 6000: 0.000629
Minibatch accuracy: 7.8%
Minibatch loss at step 6500: 0.000706
Minibatch accuracy: 10.9%
Minibatch loss at step 7000: 0.000447
Minibatch accuracy: 11.7%
Minibatch loss at step 7500: 0.0007

In [41]:
# adding one hidden layer with relu and 1024 hidden nodes
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2]))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
  logits = tf.matmul(lay2_train, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(lay2_test, weights3) + biases3)

In [42]:
num_steps = 781

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #print("processing the range of " , offset, (offset + batch_size))
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 0.179889
Minibatch accuracy: 7.8%
Minibatch loss at step 50: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 100: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 150: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 200: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 250: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 300: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 350: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 400: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 450: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 550: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 600: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 650: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 700: nan
Minibatch accuracy: 100.0%
Minibatch loss at step 750: nan
Minibatch accuracy: 100.0%
Test accuracy: 100.0%
